In [1]:
import asyncio
import json

# import libsql_experimental as libsql
import sqlite3

import pandas as pd
from tqdm import tqdm
import aiohttp

from mercatracker import api, db, logging
from mercatracker.config import Config
from mercatracker.scraper import Soup

In [44]:
config = Config().load()
conn = sqlite3.connect("/home/miguel/git/mercadona-tracker/src/mercadona_test.db")

soup = Soup(url=config.url_sitemap).request()
lastmod = soup.get_lastmod()

if lastmod != db.get_lastmod(conn):
    all_ids = soup.get_ids()
    db.write_scraped_ids(
        conn,
        (
            lastmod,
            json.dumps(all_ids),
        ),
    )
    # config["LASTMOD"] = lastmod
    # config.update(file="temp", var={"LASTMOD": lastmod})
else:
    all_ids = db.get_scraped_ids(conn, lastmod)

ids = set(all_ids) - set(db.get_processed_ids(conn, lastmod))

# products = [api.ProductSchema(id=id, params=config.params_api) for id in ids]

In [37]:
import concurrent
import itertools
with concurrent.futures.ThreadPoolExecutor(2) as executor:
        res = [executor.submit(api.ProductSchema().request(), id=id, ) for id in list(itertools.islice(ids, 10))]
        concurrent.futures.wait(res)

TypeError: ProductSchema.__init__() missing 2 required positional arguments: 'id' and 'params'

In [31]:
with concurrent.futures.ThreadPoolExecutor(2) as executor:
    res_ = executor.submit(api.ProductSchema(id=8244, params=config.params_api).request())

In [48]:
list(zip(list(itertools.islice(ids, 10)), itertools.repeat(config.params_api)))

[('8244', {'lang': 'es', 'wh': 'vlc1'}),
 ('20040', {'lang': 'es', 'wh': 'vlc1'}),
 ('59223', {'lang': 'es', 'wh': 'vlc1'}),
 ('80942', {'lang': 'es', 'wh': 'vlc1'}),
 ('7032', {'lang': 'es', 'wh': 'vlc1'}),
 ('76487', {'lang': 'es', 'wh': 'vlc1'}),
 ('67739', {'lang': 'es', 'wh': 'vlc1'}),
 ('33178', {'lang': 'es', 'wh': 'vlc1'}),
 ('34145', {'lang': 'es', 'wh': 'vlc1'}),
 ('73988', {'lang': 'es', 'wh': 'vlc1'})]

In [51]:
list(itertools.islice(ids, 10))

['8244',
 '20040',
 '59223',
 '80942',
 '7032',
 '76487',
 '67739',
 '33178',
 '34145',
 '73988']

In [59]:
from functools import partial
[partial(api.ProductSchema, id=id, params=config.params_api) for id in ids_]

[functools.partial(<class 'mercatracker.api.ProductSchema'>, id='8244', params={'lang': 'es', 'wh': 'vlc1'}),
 functools.partial(<class 'mercatracker.api.ProductSchema'>, id='20040', params={'lang': 'es', 'wh': 'vlc1'}),
 functools.partial(<class 'mercatracker.api.ProductSchema'>, id='59223', params={'lang': 'es', 'wh': 'vlc1'}),
 functools.partial(<class 'mercatracker.api.ProductSchema'>, id='80942', params={'lang': 'es', 'wh': 'vlc1'}),
 functools.partial(<class 'mercatracker.api.ProductSchema'>, id='7032', params={'lang': 'es', 'wh': 'vlc1'}),
 functools.partial(<class 'mercatracker.api.ProductSchema'>, id='76487', params={'lang': 'es', 'wh': 'vlc1'}),
 functools.partial(<class 'mercatracker.api.ProductSchema'>, id='67739', params={'lang': 'es', 'wh': 'vlc1'}),
 functools.partial(<class 'mercatracker.api.ProductSchema'>, id='33178', params={'lang': 'es', 'wh': 'vlc1'}),
 functools.partial(<class 'mercatracker.api.ProductSchema'>, id='34145', params={'lang': 'es', 'wh': 'vlc1'}),
 fu

In [68]:
ids_ = list(itertools.islice(ids, 10))
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    worker = [partial(api.ProductSchema, id=id, params=config.params_api) for id in ids_]
    results = executor.map(worker, range(0, len(ids_)))

AttributeError: 'functools.partial' object has no attribute 'request'

In [79]:
ids_ = list(itertools.islice(ids, 50))

def fetch_product_data(id, params):
    product = api.ProductSchema(id=id, params=params)
    product.request()  # Perform the request
    return product  # Return the ProductSchema instance

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    futures = [executor.submit(fetch_product_data, id, config.params_api) for id in ids_]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

In [85]:
results[0].decode()

{'id': '8244',
 'ean': '8402001023064',
 'slug': 'patatas-fritas-sabor-sal-vinagre-hacendado-paquete',
 'brand': 'Hacendado',
 'limit': 999,
 'badges': {'is_water': False, 'requires_age_check': False},
 'origin': None,
 'photos': [{'zoom': 'https://prod-mercadona.imgix.net/images/d3f7142f4a8c9bd9db796a6387539ad5.jpg?fit=crop&h=1600&w=1600',
   'regular': 'https://prod-mercadona.imgix.net/images/d3f7142f4a8c9bd9db796a6387539ad5.jpg?fit=crop&h=600&w=600',
   'thumbnail': 'https://prod-mercadona.imgix.net/images/d3f7142f4a8c9bd9db796a6387539ad5.jpg?fit=crop&h=300&w=300',
   'perspective': 2},
  {'zoom': 'https://prod-mercadona.imgix.net/images/4a0b3444a786583341599f8c209501ca.jpg?fit=crop&h=1600&w=1600',
   'regular': 'https://prod-mercadona.imgix.net/images/4a0b3444a786583341599f8c209501ca.jpg?fit=crop&h=600&w=600',
   'thumbnail': 'https://prod-mercadona.imgix.net/images/4a0b3444a786583341599f8c209501ca.jpg?fit=crop&h=300&w=300',
   'perspective': 9}],
 'status': None,
 'details': {'bra

In [84]:
valid_requests = [result for result in results if result.response.status_code == 200]

In [ ]:
for product in valid_requests:
    db.write_dump(conn, (product.id, str(product.decode()), lastmod,),)

In [78]:
db.write_dump(conn, (id, str(result.json()), lastmod,),)

{'id': '8244',
 'ean': '8402001023064',
 'slug': 'patatas-fritas-sabor-sal-vinagre-hacendado-paquete',
 'brand': 'Hacendado',
 'limit': 999,
 'badges': {'is_water': False, 'requires_age_check': False},
 'origin': None,
 'photos': [{'zoom': 'https://prod-mercadona.imgix.net/images/d3f7142f4a8c9bd9db796a6387539ad5.jpg?fit=crop&h=1600&w=1600',
   'regular': 'https://prod-mercadona.imgix.net/images/d3f7142f4a8c9bd9db796a6387539ad5.jpg?fit=crop&h=600&w=600',
   'thumbnail': 'https://prod-mercadona.imgix.net/images/d3f7142f4a8c9bd9db796a6387539ad5.jpg?fit=crop&h=300&w=300',
   'perspective': 2},
  {'zoom': 'https://prod-mercadona.imgix.net/images/4a0b3444a786583341599f8c209501ca.jpg?fit=crop&h=1600&w=1600',
   'regular': 'https://prod-mercadona.imgix.net/images/4a0b3444a786583341599f8c209501ca.jpg?fit=crop&h=600&w=600',
   'thumbnail': 'https://prod-mercadona.imgix.net/images/4a0b3444a786583341599f8c209501ca.jpg?fit=crop&h=300&w=300',
   'perspective': 9}],
 'status': None,
 'details': {'bra

In [23]:
list(itertools.islice(ids, 10))

['8244',
 '20040',
 '59223',
 '80942',
 '7032',
 '76487',
 '67739',
 '33178',
 '34145',
 '73988']

In [27]:
res[0].result()

TypeError: 'Response' object is not callable

In [3]:
async with aiohttp.ClientSession() as session:
    tasks = [product.async_post(session) for product in products]

In [4]:
id = ids.pop()
product = api.ProductSchema(id=id, params=config.params_api)

In [5]:
response = product.request_force_retry()

In [6]:
product.decode()

{'id': '22530',
 'ean': '8480000225306',
 'slug': 'pintalabios-creamy-deliplus-202-rosa-calido',
 'brand': 'Deliplus',
 'limit': 999,
 'badges': {'is_water': False, 'requires_age_check': False},
 'origin': None,
 'photos': [{'zoom': 'https://prod-mercadona.imgix.net/images/7dc65c2bbd8b4d6c5d6e4701fec637c5.jpg?fit=crop&h=1600&w=1600',
   'regular': 'https://prod-mercadona.imgix.net/images/7dc65c2bbd8b4d6c5d6e4701fec637c5.jpg?fit=crop&h=600&w=600',
   'thumbnail': 'https://prod-mercadona.imgix.net/images/7dc65c2bbd8b4d6c5d6e4701fec637c5.jpg?fit=crop&h=300&w=300',
   'perspective': 10},
  {'zoom': 'https://prod-mercadona.imgix.net/images/fb9b57e0ee18c9b5d32fae39f747dfd0.jpg?fit=crop&h=1600&w=1600',
   'regular': 'https://prod-mercadona.imgix.net/images/fb9b57e0ee18c9b5d32fae39f747dfd0.jpg?fit=crop&h=600&w=600',
   'thumbnail': 'https://prod-mercadona.imgix.net/images/fb9b57e0ee18c9b5d32fae39f747dfd0.jpg?fit=crop&h=300&w=300',
   'perspective': 9},
  {'zoom': 'https://prod-mercadona.imgix.

In [4]:
tasks = []
loop = asyncio.get_event_loop()
async with aiohttp.ClientSession() as session:
    for product in products[0:10]:
        tasks.append(product.async_post(session))

    loop.run_until_complete(asyncio.gather(*tasks))

[product.decode() for product in products]

/tmp/ipykernel_532059/4132782221.py:1: RuntimeWarning: coroutine 'ProductSchema.async_post' was never awaited
  tasks = []


RuntimeError: This event loop is already running

In [5]:
loop = asyncio.get_running_loop()
done, _ = await loop.run_until_complete(asyncio.gather(*tasks))

RuntimeError: This event loop is already running

In [10]:
N=5
tasks = []

# async with aiohttp.ClientSession() as session:
#     tasks = asyncio.Queue()
#     for product in products[0:10]:
#         tasks.put_nowait(product.async_post(session))

    # async def worker():
    #     while not tasks.empty():
    #         await tasks.get_nowait()

    # await asyncio.gather(*[worker() for _ in range(N)])

In [11]:
for product, result in zip(products, results):
    if result.status_code == 200:
        db.write_dump(
            conn,
            (
                product.id,
                str(product.async_decode()),
                lastmod,
            ),
        )

[{'id': '69884',
  'ean': '8437014515435',
  'slug': 'ensalada-jamon-compuesta-jamon-curado-queso-curado-picatostes-con-tomate-oregano-aceite-sal-vinagre-incluye-tenedor-bol',
  'brand': None,
  'limit': 999,
  'badges': {'is_water': False, 'requires_age_check': False},
  'origin': None,
  'photos': [{'zoom': 'https://prod-mercadona.imgix.net/images/16a202a352a201d77763ad9632b5c1e9.jpg?fit=crop&h=1600&w=1600',
    'regular': 'https://prod-mercadona.imgix.net/images/16a202a352a201d77763ad9632b5c1e9.jpg?fit=crop&h=600&w=600',
    'thumbnail': 'https://prod-mercadona.imgix.net/images/16a202a352a201d77763ad9632b5c1e9.jpg?fit=crop&h=300&w=300',
    'perspective': 2},
   {'zoom': 'https://prod-mercadona.imgix.net/images/fcce1641b1ba4079b59007cb613ba1d5.jpg?fit=crop&h=1600&w=1600',
    'regular': 'https://prod-mercadona.imgix.net/images/fcce1641b1ba4079b59007cb613ba1d5.jpg?fit=crop&h=600&w=600',
    'thumbnail': 'https://prod-mercadona.imgix.net/images/fcce1641b1ba4079b59007cb613ba1d5.jpg?fit

In [19]:
tasks_ = []
for product in products[0:10]:
    tasks_.append(product.async_decode())
decoded = await asyncio.gather(*tasks_)

TypeError: object dict can't be used in 'await' expression

In [20]:
tasks_

[<coroutine object ProductSchema.async_decode at 0x7fe4fa5120a0>,
 <coroutine object ProductSchema.async_decode at 0x7fe4fa513e60>,
 <coroutine object ProductSchema.async_decode at 0x7fe4fa511fc0>,
 <coroutine object ProductSchema.async_decode at 0x7fe4fa512500>,
 <coroutine object ProductSchema.async_decode at 0x7fe4fa512260>,
 <coroutine object ProductSchema.async_decode at 0x7fe4fa513ca0>,
 <coroutine object ProductSchema.async_decode at 0x7fe4fa513d80>,
 <coroutine object ProductSchema.async_decode at 0x7fe4fa5133e0>,
 <coroutine object ProductSchema.async_decode at 0x7fe4fa513680>,
 <coroutine object ProductSchema.async_decode at 0x7fe4fa513920>]

In [16]:
await products[0].async_decode().result()

/home/miguel/miniconda3/envs/merca/lib/python3.12/site-packages/pygments/formatters/terminal256.py:180: RuntimeWarning: coroutine 'ProductSchema.async_decode' was never awaited
  self.xterm_colors.append((r, g, b))


AttributeError: 'coroutine' object has no attribute 'result'

RuntimeError: asyncio.run() cannot be called from a running event loop